In [4]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2

In [5]:
# load eBeam XML file
tree = ET.parse('original/a01/a01-001/strokesz.xml')
root = tree.getroot()

In [6]:
# get first set of strokes from XML file
strokeset = root.find('StrokeSet')
min_x, min_y, max_x, max_y = 10000, 10000, 0, 0

for stroke in strokeset:
    _points = stroke.findall('Point')
    _points = np.array([[float(point.get('x')), float(point.get('y'))] for point in _points]) / 10
    _points = _points.astype(int)

    min_x = min(min_x, np.min(_points[:, 0]))
    min_y = min(min_y, np.min(_points[:, 1]))
    max_x = max(max_x, np.max(_points[:, 0]))
    max_y = max(max_y, np.max(_points[:, 1]))

min_y -= 20
min_x -= 20
max_y += 20
max_x += 20

#get each line from the transcription
transcription = root.find('Transcription')
text = transcription.find('Text')
linesprint = text.text.strip().splitlines()

In [22]:
def bound_line(line_strokes):
    if line_strokes:

        #find min_x and min_y for each line
        min_x_line = 10000
        min_y_line = 10000

        for line_stroke in line_strokes:
            min_x_line = min(min_x_line, np.min(line_stroke[:, 0]))
            min_y_line = min(min_y_line, np.min(line_stroke[:, 1]))
            
        # make a 20 offset to min_x and min_y
        min_x_line -= 20
        min_y_line -= 20
        
        # reduce min_x and min_y from each points in points
        for line_stroke in line_strokes:
            line_stroke[:, 0] -= min_x_line
            line_stroke[:, 1] -= min_y_line
    return line_strokes, min_x_line, min_y_line

In [25]:


# set threshold for x-axis difference between paragraphs
threshold = 200

# initialize variables for tracking paragraph boundaries
prev_last_point = None
curr_first_point = None
is_new_paragraph = False
line_no = 0

# initialize dictionary to store online handwriting data
line_data = {}
line_strokes = []

for stroke in strokeset:
    _points = stroke.findall('Point')

    # convert the points to a numpy array
    _points = np.array([[float(point.get('x')), float(point.get('y'))] for point in _points]) / 10
    _points = _points.astype(int)

    curr_first_point = _points[0]

    # check if this stroke is a new paragraph
    if prev_last_point is not None and curr_first_point is not None and curr_first_point[0] - prev_last_point[0] < -threshold:
        # this stroke is a new paragraph
        # add previous line strokes to the line data dictionary
        line_data[linesprint[line_no]], _, _ = bound_line(line_strokes)
        line_no += 1
        line_strokes = []

    # add current stroke to current line strokes
    line_strokes.append(_points)

    # update variables for tracking paragraph boundaries
    prev_last_point = _points[-1]

# add last line strokes to the line data dictionary
if line_strokes:
    line_data[linesprint[line_no]],_ ,_  = bound_line(line_strokes)


In [5]:
# set threshold for x-axis difference between paragraphs
threshold = 200

# initialize variables for tracking paragraph boundaries
prev_last_point = None
curr_first_point = None
is_new_paragraph = False
line_no = 0

# initialize dictionary to store online handwriting data
line_data = {}
line_strokes = []

for stroke in strokeset:
    _points = stroke.findall('Point')

    # convert the points to a numpy array
    _points = np.array([[float(point.get('x')), float(point.get('y'))] for point in _points]) / 10
    _points = _points.astype(int)

    # reduce min_x and min_y from each points in points
    _points[:, 0] -= min_x
    _points[:, 1] -= min_y

    curr_first_point = _points[0]

    # check if this stroke is a new paragraph
    if prev_last_point is not None and curr_first_point is not None:
        if curr_first_point[0] - prev_last_point[0] < -threshold:
            is_new_paragraph = True

            line_data[linesprint[line_no]] = (line_strokes)

            #update line number
            line_no += 1

        else:
            is_new_paragraph = False
            line_strokes.append(_points)

    # update variables for tracking paragraph boundaries
    prev_last_point = _points[-1]

#adding the last line
#line_no += 1
line_data[linesprint[line_no]] = line_strokes


In [6]:
# set threshold for x-axis difference between paragraphs
threshold = 200

# initialize variables for tracking paragraph boundaries
prev_last_point = None
curr_first_point = None
is_new_paragraph = False
line_no = 0

# initialize dictionary to store online handwriting data
line_data = {}
line_strokes = []

for stroke in strokeset:
    _points = stroke.findall('Point')

    # convert the points to a numpy array
    _points = np.array([[float(point.get('x')), float(point.get('y'))] for point in _points]) / 10
    _points = _points.astype(int)

    # reduce min_x and min_y from each points in points
    _points[:, 0] -= min_x
    _points[:, 1] -= min_y

    curr_first_point = _points[0]

    # check if this stroke is a new paragraph
    if prev_last_point is not None and curr_first_point is not None and curr_first_point[0] - prev_last_point[0] < -threshold:
        # this stroke is a new paragraph
        is_new_paragraph = True

        # add previous line strokes to the line data dictionary
        if line_strokes:
            line_data[linesprint[line_no]] = line_strokes
            line_no += 1
        line_strokes = []

    # add current stroke to current line strokes
    line_strokes.append(_points)

    # update variables for tracking paragraph boundaries
    prev_last_point = _points[-1]

# add last line strokes to the line data dictionary
if line_strokes:
    line_data[linesprint[line_no]] = line_strokes


In [ ]:
    def bound_line(self):
        if self.strokeset:
            # find min_x, min_y, max_x, and max_y for each line
            min_x_line = 10000
            min_y_line = 10000
            max_x_line = 0
            max_y_line = 0

            for stroke in self.strokeset:
                points = stroke.findall('Point')
                points = np.array([[float(point.get('x')), float(point.get('y'))] for point in points]) / 10
                points = points.astype(int)

                min_x_line = min(min_x_line, np.min(points[:, 0]))
                min_y_line = min(min_y_line, np.min(points[:, 1]))
                max_x_line = max(max_x_line, np.max(points[:, 0]))
                max_y_line = max(max_y_line, np.max(points[:, 1]))

                self.line_strokes.append(points)

            # add a 20 pixel offset to min_x and min_y
            self.min_x = min_x_line - 20
            self.min_y = min_y_line - 20

            # subtract min_x and min_y from each point in each stroke
            for i, stroke in enumerate(self.line_strokes):
                self.line_strokes[i][:, 0] -= self.min_x
                self.line_strokes[i][:, 1] -= self.min_y

            self.max_x = max_x_line - self.min_x
            self.max_y = max_y_line - self.min_y

In [26]:
for line, strokes in line_data.items():
    img = np.zeros((max_y - min_y, max_x - min_x, 3), np.uint8)
    for stroke in strokes:
        cv2.polylines(img, [stroke], False, (0, 255, 0), 2)
    cv2.imshow(line, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [48]:
class HandwrittenLine:
    def __init__(self, line_strokes, text):
        self.text = text
        self.line_strokes = line_strokes
        self.min_x = None
        self.min_y = None
        self.max_x = None
        self.max_y = None

        self.bound_line()

    def bound_line(self):
        # find min and max x,y coordinates for the line
        self.min_x = min(point[0] for stroke in self.line_strokes for point in stroke) - 20
        self.min_y = min(point[1] for stroke in self.line_strokes for point in stroke) - 20
        self.max_x = max(point[0] for stroke in self.line_strokes for point in stroke) + 20
        self.max_y = max(point[1] for stroke in self.line_strokes for point in stroke) + 20

        # adjust the coordinates of the points in the line so that they are relative to the bounding box
        for stroke in self.line_strokes:
            for point in stroke:
                point[0] -= self.min_x
                point[1] -= self.min_y

        # return the adjusted strokes and bounding box details
        return self.min_x, self.min_y, self.max_x, self.max_y

    def get_line_image(self):
        # initialize a blank image
        img = np.zeros((self.max_y - self.min_y, self.max_x - self.min_x, 3), np.uint8)

        # draw each stroke onto the image
        for stroke in self.line_strokes:
            cv2.polylines(img, [stroke], False, (0, 255, 0), 2)

        return img
    
    def show_line_image(self):
        cv2.imshow(self.text, self.get_line_image())
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [49]:
# make a handwritingline object using the first line in the line data dictionary key and value
line = HandwrittenLine(line_data[linesprint[1]], linesprint[1])

line.show_line_image()
